In [61]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from os import listdir
from os.path import isfile, join


from src.SimilarityArtworks import *
from src.SimilarityUsers import *
from src.ArtworksMatrix import *
from src.UsersMatrix import *
from src.UsersClustering import *
from src.AverageUser import *

In [62]:
import inspect

def inner_classes_list(cls):
    return [cls_attribute.__name__ for cls_attribute in cls.__dict__.values()
            if inspect.isclass(cls_attribute)]
            # and issubclass(cls_attribute, SimilarityFunctionInterface)]

In [63]:
def elegirDatos(path, files):
  usersDropdown = widgets.Dropdown(
      options=files,
      value='Prado_users_emotions_OnePolarity.csv',
      description='Users:',
      disabled=False,
  )
  itemsDropdown = widgets.Dropdown(
      options=files,
      value='Prado_artworks_wikidata.csv',
      description='Items:',
      disabled=False,
  )
  return usersDropdown, itemsDropdown

def cargarDatos(path, usersDropdown, itemsDropdown):
  users = pd.read_csv(path + usersDropdown.value, index_col=0)
  artworks = pd.read_csv(path + itemsDropdown.value)
  users['positive'] = users['positive'].apply(eval)
  users['negative'] = users['negative'].apply(eval)
  users['mixed'] = users['mixed'].apply(eval)
  return users, artworks

def elegirSimilitudItems(path, files, artworks):
  functions = [None]
  functions.extend(inner_classes_list(SimilarityArtworks))

  dropdowns = {}
  for col in artworks.columns:
    dropdowns[col] = widgets.Dropdown(options=functions, value=None, description=col, disabled=False)

  colorDropdown = widgets.Dropdown(options=files, value=None, description='Colors', disabled=False)

  return dropdowns, colorDropdown

def elegirSimilitudUsers(path, files, users):
  functions = [None]
  functions.extend(inner_classes_list(SimilarityUsers))

  dropdowns = {}
  for col in users.columns:
    dropdowns[col] = widgets.Dropdown(options=functions, value=None, description=col, disabled=False)
  
  return dropdowns

def elegirPesosItems(atr_dropdowns, colorDropdown):
  floatTexts = {}
  colorFloatText = None
  for key, value in atr_dropdowns.items():
    if value.value is not None:
      floatTexts[key] = widgets.FloatText(value=0.3, description=key, disabled=False)

  if colorDropdown.value is not None:
    colorFloatText = widgets.FloatText(value=0.3, description='Colors', disabled=False)
    
  return floatTexts, colorFloatText

def elegirPesosUsers(atr_dropdowns):
  floatTexts = {}
  for key, value in atr_dropdowns.items():
    if value.value is not None:
      floatTexts[key] = widgets.FloatText(value=0.3, description=key, disabled=False)
  
  demographicFloatText = widgets.FloatText(value=0.5, description='Demographic', disabled=False)
  polarityFloatText = widgets.FloatText(value=0.5, description='Polarity', disabled=False)
  
  return floatTexts, demographicFloatText, polarityFloatText

def calcularSimilitudItems(path, artworks, atr_dropdowns, float_texts):
  funcs = {x : None for x in artworks.columns}
  weights = {x : None for x in artworks.columns}

  for key, value in atr_dropdowns.items():
    funcs.update({key: value.value})

  for key, value in float_texts.items():
    weights.update({key: value.value})
  weights.update({'Colors': colorFloatText.value})

  AM = ArtworksMatrix(artworks, function_dict=funcs, weight_dict=weights, colors_path=path + colorDropdown.value)
  
  return AM.getSimilarityMatrix()

def calcularSimilitudUsers(path, users, artworks_sim, atr_dropdowns, float_texts, demographicFloatText, polarityFloatText):
  funcs = {x : None for x in users.columns}
  weights = {x : None for x in users.columns}

  for key, value in atr_dropdowns.items():
    funcs.update({key: value.value})

  for key, value in float_texts.items():
    weights.update({key: value.value})

  weights.update({'polarity' : polarityFloatText.value, 'demographic' : demographicFloatText.value})  
  
  UM = UsersMatrix(users, artworks_sim, function_dict=funcs, weight_dict=weights)
  
  return UM.getSimilarityMatrix()

def calcularClusters(users, users_sim):
  users_distance = 1 - users_sim
  labels = UsersClustering(users_sim).kMedoidsFromMatrix()
  users_clustered = users.copy()
  users_clustered['cluster'] = labels
  return users_clustered



In [64]:
data_path = 'data/'
files = [f for f in listdir(data_path) if isfile(join(data_path, f))]

usersDropdown, itemsDropdown = elegirDatos(data_path, files)
display(itemsDropdown)
display(usersDropdown)

Dropdown(description='Items:', index=8, options=('artworkColors.json', 'artworkEmotions.json', 'Clean_users.cs…

Dropdown(description='Users:', index=14, options=('artworkColors.json', 'artworkEmotions.json', 'Clean_users.c…

In [65]:
users, artworks = cargarDatos(data_path, usersDropdown, itemsDropdown)
atr_dropdowns, colorDropdown = elegirSimilitudItems(data_path, files, artworks)
for key, value in atr_dropdowns.items():
  display(value)
display(colorDropdown)

Dropdown(description='ID', options=(None, 'SimilarityArtist', 'SimilarityCategory', 'SimilarityColors'), value…

Dropdown(description='Category', options=(None, 'SimilarityArtist', 'SimilarityCategory', 'SimilarityColors'),…

Dropdown(description='Artist', options=(None, 'SimilarityArtist', 'SimilarityCategory', 'SimilarityColors'), v…

Dropdown(description='Title', options=(None, 'SimilarityArtist', 'SimilarityCategory', 'SimilarityColors'), va…

Dropdown(description='Year', options=(None, 'SimilarityArtist', 'SimilarityCategory', 'SimilarityColors'), val…

...

Dropdown(description='wd:paintingID', options=(None, 'SimilarityArtist', 'SimilarityCategory', 'SimilarityColo…

Dropdown(description='wd:paintingURL', options=(None, 'SimilarityArtist', 'SimilarityCategory', 'SimilarityCol…

Dropdown(description='wd:artistID', options=(None, 'SimilarityArtist', 'SimilarityCategory', 'SimilarityColors…

Dropdown(description='wd:artistURL', options=(None, 'SimilarityArtist', 'SimilarityCategory', 'SimilarityColor…

Dropdown(description='Colors', options=('artworkColors.json', 'artworkEmotions.json', 'Clean_users.csv', 'coun…

In [66]:
atributes_artworks = [i for i in [value.value for key, value in atr_dropdowns.items()] if i]

float_texts, colorFloatText = elegirPesosItems(atr_dropdowns, colorDropdown)
for key, value in float_texts.items():
  display(value)

if colorDropdown.value is not None:
  display(colorFloatText) 

FloatText(value=0.3, description='Category')

FloatText(value=0.3, description='Artist')

FloatText(value=0.3, description='Colors')

In [67]:
artworks_sim = calcularSimilitudItems(data_path, artworks, atr_dropdowns, float_texts)

In [49]:
atr_dropdowns = elegirSimilitudUsers(data_path, files, users)
for key, value in atr_dropdowns.items():
  display(value)

Dropdown(description='userId', options=(None, 'SimilarityAge', 'SimilarityGender', 'SimilarityCountry', 'Simil…

Dropdown(description='age', options=(None, 'SimilarityAge', 'SimilarityGender', 'SimilarityCountry', 'Similari…

Dropdown(description='gender', options=(None, 'SimilarityAge', 'SimilarityGender', 'SimilarityCountry', 'Simil…

Dropdown(description='country', options=(None, 'SimilarityAge', 'SimilarityGender', 'SimilarityCountry', 'Simi…

Dropdown(description='positive', options=(None, 'SimilarityAge', 'SimilarityGender', 'SimilarityCountry', 'Sim…

Dropdown(description='negative', options=(None, 'SimilarityAge', 'SimilarityGender', 'SimilarityCountry', 'Sim…

Dropdown(description='mixed', options=(None, 'SimilarityAge', 'SimilarityGender', 'SimilarityCountry', 'Simila…

In [50]:
atributes_users = [i for i in [value.value for key, value in atr_dropdowns.items()] if i]

float_texts, demographicFloatText, polarityFloatText = elegirPesosUsers(atr_dropdowns)
display(demographicFloatText)
display(polarityFloatText)
for key, value in float_texts.items():
  display(value) 

FloatText(value=0.5, description='Demographic')

FloatText(value=0.5, description='Polarity')

FloatText(value=0.3, description='age')

FloatText(value=0.3, description='gender')

FloatText(value=0.3, description='country')

FloatText(value=0.3, description='positive')

FloatText(value=0.3, description='negative')

FloatText(value=0.3, description='mixed')

In [51]:
users_sim = calcularSimilitudUsers(data_path, users, artworks_sim, atr_dropdowns, float_texts, demographicFloatText, polarityFloatText)

In [52]:
%%capture output
users_clustered = calcularClusters(users, users_sim)

In [57]:
AU = AverageUser(users_clustered, artworks, atributes_users, atributes_artworks)

In [59]:
explicators = AU.computeAverageUser()
explicators

,userId,age,gender,country,positive,negative,mixed,cluster
0,expl307,45-54,female,Spain,"[57726f59edc2cb3880ba76b3, 57727c95edc2cb3880e...","[57727cbaedc2cb3880e561f7, 57726e45edc2cb3880b...","[57727015edc2cb3880bceb08, 5772700eedc2cb3880b...",3
1,expl308,25-34,female,Spain,"[57727c95edc2cb3880e4d1d4, 57726f59edc2cb3880b...","[57727cbaedc2cb3880e561f7, 57726f59edc2cb3880b...","[57727cbaedc2cb3880e561f7, 577278cdedc2cb3880d...",7
2,expl309,35-44,male,Spain,"[57726e34edc2cb3880b62340, 57726f56edc2cb3880b...","[57727cbaedc2cb3880e561f7, 57726f4fedc2cb3880b...","[57726f59edc2cb3880ba76b3, 57727015edc2cb3880b...",5
3,expl310,13-17,female,Spain,"[57727c2eedc2cb3880e39405, 57726e34edc2cb3880b...","[57726f4fedc2cb3880ba0b6d, 5772752eedc2cb3880c...",[57726f76edc2cb3880baa425],0
4,expl311,35-44,female,Other,"[57726f59edc2cb3880ba76b3, 57727015edc2cb3880b...","[57727cbaedc2cb3880e561f7, 57726e37edc2cb3880b...","[57727cbaedc2cb3880e561f7, 57727071edc2cb3880b...",6
5,expl312,45-54,male,Spain,"[57726f59edc2cb3880ba76b3, 57727c95edc2cb3880e...","[57726f59edc2cb3880ba76b3, 57727cbaedc2cb3880e...","[57726e34edc2cb3880b62340, 57726f59edc2cb3880b...",4
6,expl313,25-34,male,Spain,"[57726f59edc2cb3880ba76b3, 57727c95edc2cb3880e...","[57726e37edc2cb3880b628ee, 57726f49edc2cb3880b...","[57727015edc2cb3880bceb08, 57726f59edc2cb3880b...",9
7,expl314,55-64,female,Other,"[57727c95edc2cb3880e4d1d4, 57726f59edc2cb3880b...","[57727cbaedc2cb3880e561f7, 57726f4fedc2cb3880b...","[57727015edc2cb3880bceb28, 57727cbaedc2cb3880e...",1


In [60]:
AU.printExplanation()

------------------------
Cluster:  3
	Individuos:  34
	age :
		 45-54 ( 52.94117647058824 %)
	gender :
		 female ( 70.58823529411765 %)
	country :
		 Spain ( 44.11764705882353 %)
	--Top 3  positive--
			Title:The Straw Manikin
			Title:Rest on the Flight into Egypt
			Title:St. Rufina of Seville
	--Top 3  negative--
			Title:The Surgeon
			Title:The Triumph of Death
			Title:The Three Ages of Man and Death
	--Top 3  mixed--
			Title:St. Rufina of Seville
			Title:Apparition of the Apostle Peter to Saint Peter Nolasco
			Title:The Surgeon
------------------------
Cluster:  7
	Individuos:  39
	age :
		 25-34 ( 33.33333333333333 %)
	gender :
		 female ( 71.7948717948718 %)
	country :
		 Spain ( 48.717948717948715 %)
	--Top 3  positive--
			Title:Rest on the Flight into Egypt
			Title:The Straw Manikin
			Title:St. Rufina of Seville
	--Top 3  negative--
			Title:The Surgeon
			Title:The Straw Manikin
			Title:The Extraction of the Stone Madness
	--Top 3  mixed--
			Title:The Surgeon
			Tit